# TP : Reconnaissance de visages avec l'Analyse en Composantes Principales (ACP)

## Objectifs pédagogiques

Dans ce TP, vous allez :
- Découvrir le problème de dimensionnalité en apprentissage automatique
- Appliquer l'ACP pour réduire la dimension d'un jeu de données d'images
- Interpréter les composantes principales comme des "eigenfaces"
- Utiliser l'ACP pour la compression d'images
- Intégrer l'ACP dans un pipeline de machine learning pour la reconnaissance de visages
- Évaluer l'impact de différents prétraitements (scaling, balancing) sur les performances

## 1. Importation des bibliothèques et chargement du dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# pour importer le dataset
from sklearn.datasets import fetch_lfw_people

# pour la PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# pour la catégorisation (machine learning)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline

# divers
import time
import warnings
warnings.filterwarnings('ignore')

# Configuration pour des graphiques plus lisibles
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

### Chargement du dataset LFW (Labeled Faces in the Wild)

Le dataset LFW contient des photos de visages de personnalités publiques. Pour ce TP, nous allons filtrer pour ne garder que les personnes ayant au moins 70 photos.

In [ ]:
# Chargement du dataset
# min_faces_per_person=70 : on garde seulement les personnes avec au moins 70 photos
# resize=0.4 : on réduit la taille des images pour accélérer les calculs
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

print("Dataset chargé avec succès !")
print(f"Nombre total d'images : {lfw_people.images.shape[0]}")
print(f"Taille de chaque image : {lfw_people.images.shape[1]} x {lfw_people.images.shape[2]} pixels")

### Exploration de la structure du dataset

In [ ]:
# Extraction des données
X = lfw_people.data  # Images aplaties en vecteurs
y = lfw_people.target  # Labels (identité de la personne)
target_names = lfw_people.target_names  # Noms des personnes
n_samples, n_features = X.shape
h, w = lfw_people.images.shape[1:]

print("=" * 60)
print("STRUCTURE DU DATASET")
print("=" * 60)
print(f"Nombre d'observations (images) : {n_samples}")
print(f"Nombre de features (pixels) par image : {n_features}")
print(f"Dimensions d'une image : {h} x {w} = {h*w} pixels")
print(f"\nNombre de personnes différentes : {len(target_names)}")
print(f"\nNoms des personnes : {target_names}")

# Distribution des images par personne
print("\n" + "=" * 60)
print("DISTRIBUTION DES IMAGES PAR PERSONNE")
print("=" * 60)
unique, counts = np.unique(y, return_counts=True)
for name, count in zip(target_names[unique], counts):
    print(f"{name:30s} : {count:3d} images")

### Affichage d'un échantillon d'images

In [ ]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Fonction utilitaire pour afficher une galerie d'images"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())
    plt.show()

# Sélection aléatoire d'images pour l'affichage
np.random.seed(42)
sample_indices = np.random.choice(n_samples, 12, replace=False)
sample_images = X[sample_indices]
sample_titles = [target_names[y[i]] for i in sample_indices]

print("Échantillon aléatoire de 12 visages du dataset :")
plot_gallery(sample_images, sample_titles, h, w)

## 2. Le problème de dimensionnalité

### Analyse du ratio observations/features

En machine learning, il existe une règle empirique importante : **le nombre de features devrait être inférieur à la racine carrée du nombre d'observations** ($N_{features} < \sqrt{N_{observations}}$).

Cette règle permet d'éviter le **surapprentissage** (overfitting) et assure qu'on a suffisamment de données pour estimer correctement les paramètres du modèle.

In [ ]:
print("=" * 60)
print("ANALYSE DE LA DIMENSIONNALITÉ")
print("=" * 60)
print(f"Nombre d'observations : {n_samples}")
print(f"Nombre de features : {n_features}")
print(f"\nRatio features/observations : {n_features/n_samples:.2f}")
print(f"Racine carrée du nombre d'observations : {np.sqrt(n_samples):.2f}")
print(f"\nRègle empirique : N_features < sqrt(N_observations)")
print(f"Vérification : {n_features} < {np.sqrt(n_samples):.2f} ?")

if n_features > np.sqrt(n_samples):
    print("NON - Nous avons un problème de haute dimensionnalité !")
    print(f"\nLe ratio est de {n_features/np.sqrt(n_samples):.2f} fois trop élevé.")
    print("\nSolution : Utiliser l'ACP pour réduire la dimensionnalité.")
else:
    print("OUI - La dimensionnalité est acceptable.")

# Choix du nombre de composantes
n_components_rule = int(np.sqrt(n_samples))  # Selon la règle empirique
n_components_large = 150  # Valeur plus élevée pour comparaison

print(f"\nSelon la règle empirique (sqrt(N_observations)) :")
print(f"  Nombre maximal recommandé : {n_components_rule} composantes")
print(f"\nPour ce TP, nous allons comparer DEUX approches :")
print(f"  1. ACP 'conservatrice' : {n_components_rule} composantes (respecte la règle)")
print(f"  2. ACP 'large' : {n_components_large} composantes (pour plus d'information)")
print(f"\nCette comparaison nous aidera à comprendre le trade-off entre")
print(f"respect de la règle et conservation de l'information.")

### Application de l'ACP pour réduire la dimensionnalité

In [2]:
# Création et ajustement des deux modèles PCA (instancier pca_rule et pca_large)
# Ne pas oublier d’afficher la variance expliquée et comparer

# VOTRE CODE

## 3. Analyse des résultats de l'ACP

### Nature des composantes principales

Les composantes principales extraites de visages sont souvent appelées **"eigenfaces"**. Chaque composante représente un "mode de variation" dans l'ensemble des visages.

In [ ]:
print("=" * 60)
print("INFORMATIONS SUR L'ACP (LARGE)")
print("=" * 60)

# Expliquer en quoi consiste les composantes principales ou eigenfaces : dimension, nature, type, etc.

# VOTRE CODE

### Vérification avec une transformée inverse

Nous allons :
1. Choisir une image aléatoire
2. La transformer dans l'espace ACP (réduction de dimension)
3. Appliquer la transformée inverse pour reconstruire l'image
4. Comparer l'image originale et les images reconstruites avec les deux ACP

In [ ]:
# Sélection d'une image aléatoire (appelée original_image)

# Transformation et reconstruction avec les DEUX ACP (appelées réspectivement :
# - transformed_rule (image transformée par l’ACP conservative 35 paramétes) -> mettre suffixe _rule à toutes les variables liées à cette ACP
# - reconstructed_rule (image reonconstruite - ACP conservative)
# - transformed_large (image transformée par l’ACP large 150 paramétres) -> mettre suffixe _large à toutes les variables liées à cette ACP
# - reconstructed_large (image reonconstruite - ACP large)

# Affichage comparatif

# VOTRE CODE

# Différences
# pour calculer la différence entre l’image originale et l’image reconstruite, on peut calculer :
difference_rule = np.abs(original_image - reconstructed_rule.flatten())

# afficher et comparer les différences originale/reconstruite pour les deux ACP

# VOTRE CODE

# Calcul des erreurs
# En vous inspirant du calcul des différences, calculer les MSE : « la moyenne du carré des écarts » pour chaque ACP

# VOTRE CODE

print("=" * 70)
print("COMPARAISON DES RECONSTRUCTIONS")
print("=" * 70)
print(f"\nACP {n_components_rule} composantes (règle empirique) :")
print(f"  - Variance expliquée : {pca_rule.explained_variance_ratio_.sum():.2%}")
print(f"  - Erreur (MSE) : {mse_rule:.4f}")
print(f"\nACP {n_components_large} composantes (large) :")
print(f"  - Variance expliquée : {pca_large.explained_variance_ratio_.sum():.2%}")
print(f"  - Erreur (MSE) : {mse_large:.4f}")
print(f"\nRéduction d'erreur avec {n_components_large} composantes : {(mse_rule - mse_large)/mse_rule*100:.1f}%")
print("\nObservation : Plus de composantes = meilleure reconstruction mais plus de risque de surapprentissage !")

### Reconstruction de l'échantillon initial d'images

Comparons maintenant la qualité de reconstruction sur plusieurs images avec les deux ACP.

In [ ]:
# Reconstruction des images de l'échantillon initial de 12 images sample_images avec les DEUX ACP
sample_transformed_rule = # VOTRE CODE
sample_reconstructed_rule = # VOTRE CODE

sample_transformed_large = # VOTRE CODE
sample_reconstructed_large = # VOTRE CODE

# Affichage comparatif : 
# Afficher sur une ligne l’échantillon initial de 12 images (original)
# Afficher sur une ligne en dessous (milieu) les 12 images tranformées par l’ACP conservative
# Afficher sur la dernière ligne (bas) les 12 images transformées par l’ACP large

# VOTRE CODE

plt.show()

# Calcul des MSE moyennes
mse_mean_rule = # VOTRE CODE
mse_mean_large = # VOTRE CODE

print("=" * 70)
print("MSE MOYENNE SUR L'ÉCHANTILLON")
print("=" * 70)
print(f"ACP {n_components_rule} composantes : {mse_mean_rule:.4f}")
print(f"ACP {n_components_large} composantes : {mse_mean_large:.4f}")
print(f"\nAmélioration de la reconstruction : {(mse_mean_rule - mse_mean_large)/mse_mean_rule*100:.1f}%")

## 4. Visualisation des composantes principales ("Eigenfaces")

### Image moyenne du dataset

In [ ]:
# L'image moyenne représente le "visage moyen" de tous les visages du dataset. vous pouvez y accéder par pca.mean_
# bien sûr on n’oublie pas le .reshape()
# Affichage comparatif des images moyenne de chaque ACP

# VOTRE CODE

print("L'image moyenne représente le 'visage typique' moyen de toutes les personnes du dataset.")
print("Les composantes principales capturent les variations autour de cette moyenne.")

### Les 5 premières composantes principales

Les premières composantes capturent les plus grandes variations dans les données (ex: éclairage, orientation du visage, expressions).

**Comparaison des deux ACP :**

In [ ]:
# Comparaison des 5 premières eigenfaces des deux ACP
# la liste des composantes d’une pca peut-être trouvée à pca.components_
# afficher les 5 premières eigenfaces poru chaque ACP

eigenfaces_5_first_rule = # VOTRE CODE
eigenfaces_5_first_large = # VOTRE CODE

# Création d'une figure comparative
# afficher les 5 premières eigenfaces de ACP rule sur une ligne
# afficher les 5 premières eigenfaces de ACP large sur une autre ligne
# (pour comparer 2 à 2)

# VOTRE CODE

print("=" * 70)
print("VARIANCE EXPLIQUÉE PAR LES 5 PREMIÈRES COMPOSANTES")
print("=" * 70)

# VOTRE CODE

### Les 5 dernières composantes principales

Les dernières composantes capturent des variations plus fines et du bruit par rapport aux premières composantes.

In [ ]:
# Comparaison des 5 dernières eigenfaces des deux ACP (donc n°144, 145… pour l’une, et n°31, 32,… pour l’autre)

# VOTRE CODE

# Création d'une figure comparative

# VOTRE CODE

print("=" * 70)
print("VARIANCE EXPLIQUÉE PAR LES 5 DERNIÈRES COMPOSANTES")
print("=" * 70)

# VOTRE CODE

### 15 composantes principales aléatoires de l’ACP à 150 composantes

In [ ]:
# Sélection aléatoire de 15 indices
# sélectionner et afficher les 15 eigenfaces correspondantes, avec la variance expliquée

# VOTRE CODE

## 5. Compression d'images avec l'ACP

### Synthèse des comparaisons précédentes

Avant de passer à la compression, faisons un bilan des comparaisons entre les deux ACP :

In [ ]:
print("=" * 70)
print("SYNTHÈSE : ACP {0} vs ACP {1} COMPOSANTES".format(n_components_rule, n_components_large))
print("=" * 70)

print(f"\n1. RESPECT DE LA RÈGLE EMPIRIQUE :")
print(f"   ACP {n_components_rule} : Respecte sqrt(N) = {np.sqrt(n_samples):.1f}")
print(f"   ACP {n_components_large} : Dépasse la règle ({n_components_large/np.sqrt(n_samples):.2f}x)")

print(f"\n2. VARIANCE EXPLIQUÉE :")
print(f"   ACP {n_components_rule} : {pca_rule.explained_variance_ratio_.sum():.2%}")
print(f"   ACP {n_components_large} : {pca_large.explained_variance_ratio_.sum():.2%}")
print(f"   Gain : +{(pca_large.explained_variance_ratio_.sum() - pca_rule.explained_variance_ratio_.sum())*100:.1f}%")

print(f"\n3. QUALITÉ DE RECONSTRUCTION (MSE moyenne) :")
print(f"   ACP {n_components_rule} : MSE plus élevée (moins précis)")
print(f"   ACP {n_components_large} : MSE plus faible (plus précis)")

print(f"\n4. RISQUE DE SURAPPRENTISSAGE :")
print(f"   ACP {n_components_rule} : Risque faible (suit la règle)")
print(f"   ACP {n_components_large} : Risque potentiellement plus élevé")

print("\n" + "=" * 70)
print("\nCONCLUSION INTERMÉDIAIRE :")
print("\nNous avons vu deux approches :")
print(f"  • ACP {n_components_rule} : CONSERVATRICE - respecte les règles, mais perd de l'info")
print(f"  • ACP {n_components_large} : GOURMANDE - capture plus d'info, mais risque accru")
print("\nQuelle approche est la meilleure ? Cela dépend de l'application !")
print("\nC'est pourquoi la SECTION 6 va nous montrer comment OPTIMISER")
print("   le nombre de composantes de manière SYSTÉMATIQUE et DATA-DRIVEN.")
print("\nNous allons chercher le 'sweet spot' qui maximise les performances")
print("tout en minimisant le risque de surapprentissage.")

### Reconstruction progressive d'une image

Nous allons montrer comment une image peut être reconstruite progressivement en ajoutant de plus en plus de composantes principales. C'est le principe de la **compression d'images**. Pour cela nous allons refaire des ACP avec un nombre croissants de composantes à tester (de 1 à 200) : 

In [ ]:
# Sélection d'une image pour la démonstration
# On prend la première image de notre échantillon

# VOTRE CODE

# Liste des nombres de composantes à tester
n_components_list = [1, 2, 10, 20, 30, 40, 50, 75, 100, 150, 200]

# Création de la figure


# Affichage de l'image originale


# Reconstruction avec différents nombres de composantes
for idx, n_comp in enumerate(n_components_list):
    # Créer une PCA temporaire avec n_comp composantes

    # VOTRE CODE
    
    # Transformer et reconstruire
    
    # VOTRE CODE
    
    # Calcul du taux de compression et de la variance expliquée
    
    # VOTRE CODE
    
    # Affichage
    
    # VOTRE CODE

# Masquer les axes non utilisés
for idx in range(len(n_components_list) + 1, len(axes)):
    axes[idx].axis('off')

plt.suptitle("Reconstruction progressive d'une image avec un nombre croissant de composantes principales",
             fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("Observations :")
print("- Avec 1-2 composantes : vos observations ") # compléter
print("- Avec 10-20 composantes : vos observations ") # compléter
print("- Avec 50+ composantes : vos observations ") # compléter 
print("- Avec 150+ composantes : vos observations") # compléter

## 6. Choix du nombre optimal de composantes

Dans les sections précédentes, nous avons comparé deux approches :
- **ACP conservatrice** avec {n_components_rule} composantes (selon la règle empirique)
- **ACP large** avec {n_components} composantes (pour plus d'information)

Mais comment choisir le **nombre optimal** de composantes de manière rigoureuse ?

### Courbe de variance expliquée cumulée

Cette courbe nous aide à déterminer combien de composantes sont nécessaires pour capturer une certaine proportion de la variance totale. C'est un outil essentiel pour prendre une décision **data-driven** plutôt que de se fier uniquement aux règles empiriques.

In [ ]:
# Calcul de la variance expliquée cumulée
cumulative_variance = # VOTRE CODE

# Création du graphique
plt.figure(figsize=(12, 6))
plt.plot(range(1, n_components_large + 1), cumulative_variance, 'b-', linewidth=2)
plt.xlabel('Nombre de composantes principales', fontsize=12)
plt.ylabel('Variance expliquée cumulée', fontsize=12)
plt.title('Variance expliquée cumulée en fonction du nombre de composantes principales', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Ajout de lignes horizontales pour différents seuils
thresholds = [0.25, 0.5, 0.75, 0.9, 0.95]
colors = ['red', 'orange', 'yellow', 'green', 'blue']
for threshold, color in zip(thresholds, colors):
    plt.axhline(y=threshold, color=color, linestyle='--', alpha=0.5, 
                label=f'{threshold:.0%} variance')

plt.legend(loc='lower right')
plt.xlim(0, n_components_large)
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

print(f"Variance totale expliquée par {n_components_large} composantes : {cumulative_variance[-1]:.2%}")

### Nombre de composantes nécessaires pour différents seuils de variance

In [ ]:
def find_n_components_for_variance(cumulative_variance, threshold):
    """Trouve le nombre minimum de composantes pour atteindre un seuil de variance"""
    # Vérifier si le seuil est atteignable
    if threshold > cumulative_variance[-1]:
        # Retourner le nombre total de composantes si le seuil est trop élevé
        return len(cumulative_variance)
    return np.argmax(cumulative_variance >= threshold) + 1

print("=" * 70)
print("NOMBRE DE COMPOSANTES NÉCESSAIRES POUR DIFFÉRENTS SEUILS DE VARIANCE")
print("=" * 70)

variance_thresholds = [0.25, 0.50, 0.75, 0.90, 0.95]
results = []

for threshold in variance_thresholds:
    n_comp_needed = find_n_components_for_variance(cumulative_variance, threshold)
    variance_reached = cumulative_variance[n_comp_needed - 1]
    compression = n_features / n_comp_needed
    results.append((threshold, n_comp_needed, variance_reached, compression))
    
    print(f"\n{threshold:.0%} de variance :")
    print(f"  → {n_comp_needed} composantes nécessaires")
    print(f"  → Variance réellement atteinte : {variance_reached:.2%}")
    
    # Indiquer si le seuil n'a pas été atteint
    if variance_reached < threshold:
        print(f"!!!!!!!!!!!!!!!!!!!!!! Seuil non atteint ! Maximum disponible : {cumulative_variance[-1]:.2%}")
    
    print(f"  → Taux de compression : {compression:.1f}x")
    print(f"  → Réduction de dimensionnalité : {n_features} → {n_comp_needed} "
          f"(-{100*(1-n_comp_needed/n_features):.1f}%)")

print("\n" + "=" * 70)
print("\nInterprétation :")
print("- Plus on veut conserver de variance, plus il faut de composantes")
print("- Un bon compromis se situe souvent entre 80% et 95% de variance")
print("- Au-delà de 95%, on ajoute surtout du bruit")

## 7. Introduction au Machine Learning : Classification de visages

### Préparation des données

Nous allons maintenant utiliser l'ACP comme étape de prétraitement pour un problème de **classification supervisée** : reconnaître l'identité d'une personne à partir de son visage.

Nous reprendrons ce code en détail en cours, pour le moment contentez vous d’exécuter les cellules, et essayer de comprendre ce qu’il se passe.

In [ ]:
print("=" * 70)
print("PRÉPARATION DES ENSEMBLES D'ENTRAÎNEMENT ET DE TEST")
print("=" * 70)

# Division en ensembles d'entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nEnsemble d'entraînement : {X_train.shape[0]} images")
print(f"Ensemble de test : {X_test.shape[0]} images")
print(f"\nNombre de features : {X_train.shape[1]}")
print(f"Nombre de classes (personnes) : {len(np.unique(y))}")

# Vérification de la distribution des classes
print("\nDistribution des classes dans l'ensemble de test :")
unique_test, counts_test = np.unique(y_test, return_counts=True)
for name, count in zip(target_names[unique_test], counts_test):
    print(f"  {name:30s} : {count:2d} images")

### Entraînement sur les images originales (sans ACP)

Commençons par entraîner un modèle directement sur les images originales pour avoir une référence.

In [ ]:
print("=" * 70)
print("ENTRAÎNEMENT SUR LES IMAGES ORIGINALES (SANS ACP)")
print("=" * 70)

# Création et entraînement du modèle SVM
print("\nEntraînement en cours...")
start_time = time.time()

clf_original = SVC(kernel='rbf', gamma='scale', C=1.0, random_state=42)
clf_original.fit(X_train, y_train)

train_time_original = time.time() - start_time

# Prédiction et évaluation
y_pred_original = clf_original.predict(X_test)
accuracy_original = accuracy_score(y_test, y_pred_original)

print(f"✓ Entraînement terminé en {train_time_original:.2f} secondes")
print(f"\nPerformances sur l'ensemble de test :")
print(f"  Accuracy : {accuracy_original:.2%}")
print(f"\nRapport de classification détaillé :")
print(classification_report(y_test, y_pred_original, target_names=target_names))

### Entraînement sur les images transformées par ACP

Maintenant, appliquons l'ACP avant l'entraînement pour réduire la dimensionnalité.

In [ ]:
print("=" * 70)
print(f"ENTRAÎNEMENT AVEC ACP ({n_components_large} COMPOSANTES)")
print("=" * 70)

# Application de l'ACP
print("\nApplication de l'ACP...")
pca_ml = PCA(n_components=n_components_large, whiten=False, svd_solver='randomized')
X_train_pca = pca_ml.fit_transform(X_train)
X_test_pca = pca_ml.transform(X_test)

print(f"Dimensions après ACP :")
print(f"  Train : {X_train_pca.shape}")
print(f"  Test : {X_test_pca.shape}")
print(f"  Variance expliquée : {pca_ml.explained_variance_ratio_.sum():.2%}")

# Entraînement du modèle
print("\nEntraînement en cours...")
start_time = time.time()

clf_pca = SVC(kernel='rbf', gamma='scale', C=1.0, random_state=42)
clf_pca.fit(X_train_pca, y_train)

train_time_pca = time.time() - start_time

# Prédiction et évaluation
y_pred_pca = clf_pca.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred_pca)

print(f"✓ Entraînement terminé en {train_time_pca:.2f} secondes")
print(f"\nPerformances sur l'ensemble de test :")
print(f"  Accuracy : {accuracy_pca:.2%}")
print(f"\nRapport de classification détaillé :")
print(classification_report(y_test, y_pred_pca, target_names=target_names))

### Comparaison : Avec vs Sans ACP

In [ ]:
print("=" * 70)
print("COMPARAISON : AVEC VS SANS ACP")
print("=" * 70)

print(f"\n{'Métrique':<30} {'Sans ACP':>15} {'Avec ACP':>15} {'Différence':>15}")
print("-" * 70)

print(f"{'Nombre de features':<30} {X_train.shape[1]:>15} {X_train_pca.shape[1]:>15} "
      f"{X_train_pca.shape[1] - X_train.shape[1]:>15}")

print(f"{'Temps d’entraînement (s)':<30} {train_time_original:>15.2f} {train_time_pca:>15.2f} " 
      f"{train_time_pca - train_time_original:>15.2f}")

speedup = train_time_original / train_time_pca
print(f"{'Accélération':<30} {'':<15} {f'{speedup:.2f}x':>15} {'':>15}")

print(f"{'Accuracy':<30} {accuracy_original:>14.2%} {accuracy_pca:>14.2%} "
      f"{accuracy_pca - accuracy_original:>+14.2%}")

print("\n" + "=" * 70)
print("\nObservations :")
print(f"- Réduction de dimensionnalité : {n_features} → {n_components_large} "
      f"({100*(1-n_components_large/n_features):.1f}% de réduction)")
print(f"- L'entraînement est {speedup:.1f}x plus rapide avec l'ACP")

if accuracy_pca > accuracy_original:
    print(f"- L'accuracy est MEILLEURE avec l'ACP (+{100*(accuracy_pca-accuracy_original):.2f}%)")
    print("- L'ACP a permis d'éliminer du bruit et d'améliorer la généralisation")
elif accuracy_pca < accuracy_original:
    print(f"- L'accuracy est légèrement INFÉRIEURE avec l'ACP ({100*(accuracy_pca-accuracy_original):.2f}%)")
    print("- Un compromis acceptable pour un gain important en vitesse")
else:
    print("- L'accuracy est IDENTIQUE avec et sans ACP")
    print("- L'ACP n'a pas fait perdre d'information utile")

## 8. Optimisation du nombre de composantes principales

### Recherche du nombre optimal de composantes

Testons différents nombres de composantes pour trouver le meilleur compromis entre vitesse et performance.

In [ ]:
print("=" * 70)
print("RECHERCHE DU NOMBRE OPTIMAL DE COMPOSANTES PRINCIPALES")
print("=" * 70)

# Différentes valeurs à tester
n_components_to_test = [10, 25, 50, 75, 100, 125, 150, 175, 200]

results_optimization = []

print("\nEntraînement en cours pour différentes valeurs de n_components...\n")

for n_comp in n_components_to_test:
    # ACP
    pca_temp = PCA(n_components=n_comp, whiten=False, svd_solver='randomized')
    X_train_temp = pca_temp.fit_transform(X_train)
    X_test_temp = pca_temp.transform(X_test)
    
    # Entraînement
    start = time.time()
    clf_temp = SVC(kernel='rbf', gamma='scale', C=1.0, random_state=42)
    clf_temp.fit(X_train_temp, y_train)
    train_time = time.time() - start
    
    # Évaluation
    y_pred_temp = clf_temp.predict(X_test_temp)
    acc = accuracy_score(y_test, y_pred_temp)
    var_explained = pca_temp.explained_variance_ratio_.sum()
    
    results_optimization.append({
        'n_components': n_comp,
        'accuracy': acc,
        'train_time': train_time,
        'variance_explained': var_explained
    })
    
    print(f"n_components={n_comp:3d} | Accuracy={acc:.2%} | "
          f"Time={train_time:.2f}s | Variance={var_explained:.2%}")

print("\n✓ Optimisation terminée")

### Visualisation des résultats d'optimisation

In [ ]:
# Extraction des résultats
n_comps = [r['n_components'] for r in results_optimization]
accuracies = [r['accuracy'] for r in results_optimization]
train_times = [r['train_time'] for r in results_optimization]
variances = [r['variance_explained'] for r in results_optimization]

# Création des graphiques
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Accuracy vs n_components
axes[0, 0].plot(n_comps, accuracies, 'o-', linewidth=2, markersize=8, color='blue')
axes[0, 0].set_xlabel('Nombre de composantes principales', fontsize=11)
axes[0, 0].set_ylabel('Accuracy', fontsize=11)
axes[0, 0].set_title('Performance du modèle', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
best_idx = np.argmax(accuracies)
axes[0, 0].axvline(x=n_comps[best_idx], color='red', linestyle='--', alpha=0.5)
axes[0, 0].annotate(f'Max: {accuracies[best_idx]:.2%}\n@ {n_comps[best_idx]} comp.',
                    xy=(n_comps[best_idx], accuracies[best_idx]),
                    xytext=(10, -20), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.7),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# 2. Train time vs n_components
axes[0, 1].plot(n_comps, train_times, 'o-', linewidth=2, markersize=8, color='green')
axes[0, 1].set_xlabel('Nombre de composantes principales', fontsize=11)
axes[0, 1].set_ylabel('Temps d\'entraînement (s)', fontsize=11)
axes[0, 1].set_title('Temps d\'entraînement', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Variance expliquée vs n_components
axes[1, 0].plot(n_comps, variances, 'o-', linewidth=2, markersize=8, color='orange')
axes[1, 0].set_xlabel('Nombre de composantes principales', fontsize=11)
axes[1, 0].set_ylabel('Variance expliquée', fontsize=11)
axes[1, 0].set_title('Variance capturée', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0.9, color='red', linestyle='--', alpha=0.5, label='90%')
axes[1, 0].legend()

# 4. Trade-off Accuracy vs Train time
scatter = axes[1, 1].scatter(train_times, accuracies, c=n_comps, s=200, 
                             cmap='viridis', alpha=0.7, edgecolors='black')
axes[1, 1].set_xlabel('Temps d\'entraînement (s)', fontsize=11)
axes[1, 1].set_ylabel('Accuracy', fontsize=11)
axes[1, 1].set_title('Trade-off Performance vs Vitesse', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
cbar = plt.colorbar(scatter, ax=axes[1, 1])
cbar.set_label('Nombre de composantes', fontsize=10)

# Annotation du point optimal
axes[1, 1].scatter([train_times[best_idx]], [accuracies[best_idx]], 
                   s=300, facecolors='none', edgecolors='red', linewidths=3)

plt.tight_layout()
plt.show()

print(f"\nMeilleur résultat :")
print(f"  - {n_comps[best_idx]} composantes principales")
print(f"  - Accuracy : {accuracies[best_idx]:.2%}")
print(f"  - Temps d'entraînement : {train_times[best_idx]:.2f}s")
print(f"  p Variance expliquée : {variances[best_idx]:.2%}")

## 9. Impact du scaling et du balancing

### Effet du scaling (standardisation)

Le **scaling** consiste à centrer et réduire les données (moyenne=0, écart-type=1). C'est important pour de nombreux algorithmes de ML.

In [ ]:
print("=" * 70)
print("EFFET DU SCALING SUR LES PERFORMANCES")
print("=" * 70)

# Sans scaling (déjà fait précédemment)
print("\nRésultats SANS scaling (référence) :")
print(f"  Accuracy : {accuracy_pca:.2%}")

# Avec scaling
print("\nEntraînement AVEC scaling...")

# Standardisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ACP sur données standardisées
pca_scaled = PCA(n_components=n_components_large, whiten=False, svd_solver='randomized')
X_train_pca_scaled = pca_scaled.fit_transform(X_train_scaled)
X_test_pca_scaled = pca_scaled.transform(X_test_scaled)

# Entraînement
start_time = time.time()
clf_scaled = SVC(kernel='rbf', gamma='scale', C=1.0, random_state=42)
clf_scaled.fit(X_train_pca_scaled, y_train)
train_time_scaled = time.time() - start_time

# Évaluation
y_pred_scaled = clf_scaled.predict(X_test_pca_scaled)
accuracy_scaled = accuracy_score(y_test, y_pred_scaled)

print(f"Entraînement terminé en {train_time_scaled:.2f} secondes")
print(f"\nRésultats AVEC scaling :")
print(f"  Accuracy : {accuracy_scaled:.2%}")

print("\n" + "-" * 70)
print("COMPARAISON :")
print(f"  Différence d'accuracy : {accuracy_scaled - accuracy_pca:+.2%}")
print(f"  Différence de temps : {train_time_scaled - train_time_pca:+.2f}s")

if accuracy_scaled > accuracy_pca:
    print("\nLe scaling AMÉLIORE les performances !")
elif accuracy_scaled < accuracy_pca:
    print("\nLe scaling DIMINUE légèrement les performances.")
else:
    print("\nLe scaling n'a PAS d'effet significatif.")

print("\nRapport de classification avec scaling :")
print(classification_report(y_test, y_pred_scaled, target_names=target_names))

### Effet du balancing (équilibrage des classes)

Le **balancing** vise à donner le même poids à chaque classe lors de l'entraînement, ce qui est utile quand certaines classes sont sous-représentées.

In [ ]:
print("=" * 70)
print("ANALYSE DU DÉSÉQUILIBRE DES CLASSES")
print("=" * 70)

# Analyse de la distribution
unique_train, counts_train = np.unique(y_train, return_counts=True)

print("\nDistribution des classes dans l'ensemble d'entraînement :")
for name, count in zip(target_names[unique_train], counts_train):
    print(f"  {name:30s} : {count:3d} images ({100*count/len(y_train):.1f}%)")

# Calcul du déséquilibre
max_count = counts_train.max()
min_count = counts_train.min()
imbalance_ratio = max_count / min_count

print(f"\nRatio de déséquilibre : {imbalance_ratio:.2f}")
print(f"Classe la plus représentée : {counts_train.max()} images")
print(f"Classe la moins représentée : {counts_train.min()} images")

if imbalance_ratio > 2:
    print("\nDéséquilibre significatif détecté ! Le balancing peut aider.")
else:
    print("\nDistribution relativement équilibrée.")

In [ ]:
print("=" * 70)
print("EFFET DU BALANCING SUR LES PERFORMANCES")
print("=" * 70)

# Avec balancing (class_weight='balanced')
print("\nEntraînement AVEC balancing...")

start_time = time.time()
clf_balanced = SVC(kernel='rbf', gamma='scale', C=1.0, 
                   class_weight='balanced', random_state=42)
clf_balanced.fit(X_train_pca_scaled, y_train)
train_time_balanced = time.time() - start_time

# Évaluation
y_pred_balanced = clf_balanced.predict(X_test_pca_scaled)
accuracy_balanced = accuracy_score(y_test, y_pred_balanced)

print(f"Entraînement terminé en {train_time_balanced:.2f} secondes")
print(f"\nRésultats AVEC balancing :")
print(f"  Accuracy : {accuracy_balanced:.2%}")

print("\n" + "-" * 70)
print("COMPARAISON :")
print(f"  Sans balancing : {accuracy_scaled:.2%}")
print(f"  Avec balancing : {accuracy_balanced:.2%}")
print(f"  Différence : {accuracy_balanced - accuracy_scaled:+.2%}")

if accuracy_balanced > accuracy_scaled:
    print("\nLe balancing AMÉLIORE les performances !")
    print("   Les classes minoritaires sont mieux reconnues.")
elif accuracy_balanced < accuracy_scaled:
    print("\nLe balancing DIMINUE légèrement les performances globales.")
    print("   Mais il peut améliorer les performances sur les classes minoritaires.")
else:
    print("\nLe balancing n'a PAS d'effet significatif sur l'accuracy globale.")

print("\nRapport de classification avec balancing :")
print(classification_report(y_test, y_pred_balanced, target_names=target_names))

### Comparaison finale de toutes les configurations

In [ ]:
print("=" * 80)
print("TABLEAU RÉCAPITULATIF DE TOUTES LES CONFIGURATIONS")
print("=" * 80)

configurations = [
    ('Sans ACP (référence)', accuracy_original, train_time_original, n_features),
    (f'Avec ACP ({n_components_large} comp.)', accuracy_pca, train_time_pca, n_components_large),
    ('ACP + Scaling', accuracy_scaled, train_time_scaled, n_components_large),
    ('ACP + Scaling + Balancing', accuracy_balanced, train_time_balanced, n_components_large),
]

print(f"\n{'Configuration':<30} {'Accuracy':>12} {'Temps (s)':>12} {'Features':>12}")
print("-" * 80)

for config_name, acc, time_val, n_feat in configurations:
    print(f"{config_name:<30} {acc:>11.2%} {time_val:>11.2f} {n_feat:>12}")

print("\n" + "=" * 80)

# Visualisation graphique
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

config_names = [c[0] for c in configurations]
accuracies_all = [c[1] for c in configurations]
times_all = [c[2] for c in configurations]

# Graphique des accuracies
colors = ['red', 'blue', 'green', 'purple']
bars1 = ax1.bar(range(len(config_names)), accuracies_all, color=colors, alpha=0.7)
ax1.set_xticks(range(len(config_names)))
ax1.set_xticklabels(config_names, rotation=45, ha='right')
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('Comparaison des performances (Accuracy)', fontsize=13, fontweight='bold')
ax1.set_ylim([min(accuracies_all) * 0.95, max(accuracies_all) * 1.02])
ax1.grid(axis='y', alpha=0.3)

# Ajout des valeurs sur les barres
for bar, acc in zip(bars1, accuracies_all):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{acc:.2%}', ha='center', va='bottom', fontsize=10)

# Graphique des temps
bars2 = ax2.bar(range(len(config_names)), times_all, color=colors, alpha=0.7)
ax2.set_xticks(range(len(config_names)))
ax2.set_xticklabels(config_names, rotation=45, ha='right')
ax2.set_ylabel('Temps d\'entraînement (s)', fontsize=12)
ax2.set_title('Comparaison des temps d\'entraînement', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Ajout des valeurs sur les barres
for bar, time_val in zip(bars2, times_all):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{time_val:.2f}s', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# Conclusions
best_config_idx = np.argmax(accuracies_all)
fastest_config_idx = np.argmin(times_all)

print("\nCONCLUSIONS :")
print(f"\n1. Meilleure configuration (accuracy) : {config_names[best_config_idx]}")
print(f"   Accuracy : {accuracies_all[best_config_idx]:.2%}")
print(f"   Temps : {times_all[best_config_idx]:.2f}s")

print(f"\n2. Configuration la plus rapide : {config_names[fastest_config_idx]}")
print(f"   Temps : {times_all[fastest_config_idx]:.2f}s")
print(f"   Accuracy : {accuracies_all[fastest_config_idx]:.2%}")

speedup_final = times_all[0] / times_all[1]
print(f"\n3. Gain de vitesse avec l'ACP : {speedup_final:.1f}x plus rapide")

print("\n4. Impact des prétraitements :")
print(f"   - Scaling : {accuracies_all[2] - accuracies_all[1]:+.2%}")
print(f"   - Balancing : {accuracies_all[3] - accuracies_all[2]:+.2%}")
print(f"   - Total (ACP+Scaling+Balancing) : {accuracies_all[3] - accuracies_all[0]:+.2%} vs référence")

## Conclusion du TP

### Ce que nous avons appris :

1. **Problème de dimensionnalité** : Trop de features par rapport au nombre d'observations peut nuire aux performances et ralentir l'entraînement.

2. **L'ACP comme solution** : La réduction de dimensionnalité via ACP permet de :
   - Accélérer significativement l'entraînement
   - Réduire le surapprentissage
   - Parfois même améliorer les performances

3. **Eigenfaces** : Les composantes principales représentent des "visages types" qui capturent les variations principales dans les données.

4. **Compression d'images** : L'ACP peut être utilisée pour compresser des images en ne gardant que les composantes les plus importantes.

5. **Optimisation** : Le choix du nombre de composantes est un compromis entre :
   - La quantité d'information conservée (variance expliquée)
   - La vitesse de calcul
   - Les performances du modèle

6. **Prétraitements** :
   - Le **scaling** peut améliorer les performances en normalisant les données
   - Le **balancing** aide quand les classes sont déséquilibrées

### Points clés à retenir :

- L'ACP est une technique de **réduction de dimensionnalité linéaire**
- Elle trouve les directions de variance maximale dans les données
- C'est un outil puissant pour la **visualisation**, la **compression** et l'**accélération** des algorithmes de ML
- Elle doit être appliquée sur l'ensemble d'entraînement et les mêmes transformations appliquées au test
- Le nombre de composantes est un hyperparamètre à optimiser